In [159]:
import pickle
from docx import Document
import os 
from gensim.summarization import keywords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from tf_idf_model import bow_extractor,display_features,build_df,build_idf,build_idf_diag_matrix
from tf_idf_model import calculate_tfidf
import operator
import numpy as np
import copy
from statistics import mean
import pandas as pd

In [54]:
data_path = 'data/processed_docs.p'
names,docs_list = pickle.load(open( data_path, "rb" ))

In [55]:
print(docs_list[0][1])

['1.', 'the', 'international', 'monetary', 'and', 'financial', 'committee', 'held', 'it', 'inaugural_meeting', 'in', 'washington', ',', 'd.c.', 'on', 'april', '16', ',', '2000', ',', 'under', 'the', 'chairmanship', 'of', 'mr._gordon_brown', ',', 'chancellor', 'of', 'the', 'exchequér', 'of', 'the', 'united_kingdom', '.']


#### First of all, try use gensim

In [56]:
pos_filters = ('NN','NNS','NNP','NNPS','JJ','JJR','JJS')

In [57]:
doc = docs_list[-1]
p_list = [' '.join(p) for p in doc]
text= ' '.join(p_list)
ranks = keywords(text,split=True,ratio=1,scores=True,
                 lemmatize=False,
                 pos_filter=pos_filters)

In [58]:
r = [(v,d[0]) for v,d in ranks]

In [59]:
print(names[-1])
print(r[:20])

IMFC_37.docx
[('global growth', 0.16515996574044764), ('including', 0.14031533956094744), ('risk', 0.13599208221354295), ('sustain', 0.13380213097688948), ('sustainable', 0.13380213097688948), ('sustainability', 0.13380213097688948), ('work', 0.12316852310730401), ('working', 0.12316852310730401), ('advance policy', 0.11554694692409254), ('enhance resilience', 0.11024898119097234), ('low country', 0.10783852472832718), ('developing', 0.10748106056649637), ('support', 0.10737695864147381), ('supporting', 0.10737695864147381), ('share', 0.10215409515323824), ('competition', 0.098663019896403914), ('debt threaten', 0.097484324227672031), ('proliferation financing', 0.095731117096001683), ('enhancing debt_transparency', 0.093564332880550355), ('enhanced engagement', 0.093564332880550355)]


- so basically it does not work that well, a lot technology related and ebt related term did not show up 
- now let's multiply this by tfidf 

#### Calculate TFIDF

In [60]:
data_path = '../3_general_word_frequency/data/processed_docs.p'
names,docs = pickle.load(open( data_path, "rb" ))

In [64]:
bow_transformer,features = bow_extractor(docs)
shape = features.shape
vocabs = bow_transformer.get_feature_names()
tf = features.todense()

In [66]:
df = build_df(tf)
idf = build_idf(docs,df)

In [76]:
voc2idf = {v:i for v,i in zip(vocabs,idf)}
sorted_x = sorted(voc2idf.items(),key=operator.itemgetter(1),reverse=True)

calculate tfidf

In [128]:
idf_matrix = np.repeat(idf,shape[0],axis=0).reshape(shape[1],shape[0]).T
tfidf = np.multiply(tf,idf_matrix)


In [181]:
tfidf = np.asarray(tfidf)

In [183]:
tfidf_doc37 = copy.deepcopy(tfidf[36,:])
#tfidf_doc37 = np.squeeze(np.asarray(tfidf_doc37))

In [135]:
voc2tfidf = {v:i for v,i in zip(vocabs,tfidf_doc37)}

merge into textrank

In [157]:
data = []
for k,v in r:
    k = k.split()
    i_idf = [voc2idf.get(i,0.0) for i in k]
    i_idf = mean(i_idf)
    i_tfidf = [voc2tfidf.get(i,0.0) for i in k]
    i_tfidf = mean(i_tfidf)
    data.append((' '.join(k),v,i_idf,i_tfidf))

In [160]:
_df = pd.DataFrame(data, columns=['vocab','text_rank','idf','tfidf'])

In [161]:
_df.head()

,vocab,text_rank,idf,tfidf
0,global growth,0.165160,0.000000,0.000000
1,including,0.140315,0.026668,0.160009
2,risk,0.135992,0.111226,0.556128
3,sustain,0.133802,0.998529,0.998529
4,sustainable,0.133802,0.171850,1.031102


In [162]:
_df['text_rank_idf'] = _df['text_rank']*_df['idf']

In [163]:
_df['text_rank_tfidf'] = _df['text_rank']*_df['tfidf']

In [164]:
_df.head()

,vocab,text_rank,idf,tfidf,text_rank_idf,text_rank_tfidf
0,global growth,0.165160,0.000000,0.000000,0.000000,0.000000
1,including,0.140315,0.026668,0.160009,0.003742,0.022452
2,risk,0.135992,0.111226,0.556128,0.015126,0.075629
3,sustain,0.133802,0.998529,0.998529,0.133605,0.133605
4,sustainable,0.133802,0.171850,1.031102,0.022994,0.137964


In [174]:
result = _df.sort_values('text_rank_tfidf',ascending=False)

In [178]:
result.head(10)

,vocab,text_rank,idf,tfidf,text_rank_idf,text_rank_tfidf
28,lic,0.085698,2.251292,4.502584,0.192931,0.385862
21,inclusion,0.089964,1.239691,3.719073,0.111527,0.334581
27,lics,0.085698,1.691676,3.383352,0.144973,0.289947
18,enhancing debt_transparency,0.093564,1.607365,3.079584,0.150392,0.288139
48,target,0.067770,1.335001,4.005003,0.090473,0.271418
130,technology,0.044545,1.845827,5.537480,0.082223,0.246668
14,share,0.102154,0.459532,2.297662,0.046943,0.234716
16,debt threaten,0.097484,1.574019,2.083016,0.153442,0.203061
35,subdued productivity,0.076691,1.624910,2.623439,0.124617,0.201195
32,tackle shared challenge,0.082547,0.944793,2.382865,0.077990,0.196699
